In [1]:
# ionosphere dataset
# small numerical values.. withing the same scale..
# Good for NN

In [2]:
# SGD LR = LR/[1+ decay * epoch] so that with increasing epoch number.. the LR gets reduced.. 

In [3]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data -O ionosphere.csv

--2018-02-06 04:38:29--  http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76467 (75K) [text/plain]
Saving to: ‘ionosphere.csv’

100%[======================================>] 76,467      --.-K/s   in 0.1s    

2018-02-06 04:38:30 (761 KB/s) - ‘ionosphere.csv’ saved [76467/76467]



In [4]:
# Time Based Learning Rate Decay
from pandas import read_csv
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataframe = read_csv("ionosphere.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:34].astype(float)
Y = dataset[:,34]

Using TensorFlow backend.


In [5]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
# create model
model = Sequential()
model.add(Dense(34, input_dim=34, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
epochs = 50
learning_rate = 0.1
decay_rate = learning_rate / epochs # define decay
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False) # include decay in the model
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [6]:
# Fit the model
model.fit(X, Y, validation_split=0.33, epochs=epochs, batch_size=28, verbose=2)

Train on 235 samples, validate on 116 samples
Epoch 1/50
0s - loss: 0.6813 - acc: 0.6468 - val_loss: 0.6374 - val_acc: 0.8621
Epoch 2/50
0s - loss: 0.6367 - acc: 0.7319 - val_loss: 0.5288 - val_acc: 0.8276
Epoch 3/50
0s - loss: 0.5578 - acc: 0.8213 - val_loss: 0.4743 - val_acc: 0.8534
Epoch 4/50
0s - loss: 0.4670 - acc: 0.8383 - val_loss: 0.4407 - val_acc: 0.9224
Epoch 5/50
0s - loss: 0.3832 - acc: 0.8681 - val_loss: 0.2767 - val_acc: 0.9483
Epoch 6/50
0s - loss: 0.3159 - acc: 0.8809 - val_loss: 0.3918 - val_acc: 0.8879
Epoch 7/50
0s - loss: 0.2756 - acc: 0.9106 - val_loss: 0.2287 - val_acc: 0.9483
Epoch 8/50
0s - loss: 0.2405 - acc: 0.9106 - val_loss: 0.1451 - val_acc: 0.9569
Epoch 9/50
0s - loss: 0.2443 - acc: 0.9106 - val_loss: 0.2134 - val_acc: 0.9569
Epoch 10/50
0s - loss: 0.2011 - acc: 0.9149 - val_loss: 0.2581 - val_acc: 0.9224
Epoch 11/50
0s - loss: 0.1911 - acc: 0.9234 - val_loss: 0.1929 - val_acc: 0.9397
Epoch 12/50
0s - loss: 0.1716 - acc: 0.9404 - val_loss: 0.1150 - val_acc

In [7]:
# the above is time based LR
# time as in .. with increasing time.. epoch.. the LR decreases..

In [11]:
#The LearningRateScheduler callback allows us to defne a function to call that takes the epoch number as an argument and returns the LR for use in SGD
from keras.callbacks import LearningRateScheduler
import math


# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

# Compile model
# with new parameters for the sgd
sgd = SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
# learning schedule callback
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [12]:
# Fit the model
model.fit(X, Y, validation_split=0.33, epochs=50, batch_size=28, callbacks=callbacks_list,verbose=2)

Train on 235 samples, validate on 116 samples
Epoch 1/50
0s - loss: 0.0577 - acc: 0.9830 - val_loss: 0.0944 - val_acc: 0.9914
Epoch 2/50
0s - loss: 0.0643 - acc: 0.9787 - val_loss: 0.0690 - val_acc: 0.9914
Epoch 3/50
0s - loss: 0.0841 - acc: 0.9702 - val_loss: 0.0525 - val_acc: 0.9828
Epoch 4/50
0s - loss: 0.0825 - acc: 0.9745 - val_loss: 0.0525 - val_acc: 0.9828
Epoch 5/50
0s - loss: 0.0677 - acc: 0.9787 - val_loss: 0.1549 - val_acc: 0.9569
Epoch 6/50
0s - loss: 0.0613 - acc: 0.9830 - val_loss: 0.0602 - val_acc: 0.9914
Epoch 7/50
0s - loss: 0.0522 - acc: 0.9915 - val_loss: 0.0712 - val_acc: 0.9914
Epoch 8/50
0s - loss: 0.0457 - acc: 0.9957 - val_loss: 0.0831 - val_acc: 0.9741
Epoch 9/50
0s - loss: 0.0440 - acc: 0.9957 - val_loss: 0.0591 - val_acc: 0.9914
Epoch 10/50
0s - loss: 0.0413 - acc: 0.9915 - val_loss: 0.0679 - val_acc: 0.9914
Epoch 11/50
0s - loss: 0.0394 - acc: 0.9915 - val_loss: 0.0796 - val_acc: 0.9828
Epoch 12/50
0s - loss: 0.0369 - acc: 0.9915 - val_loss: 0.0642 - val_acc